In [1]:
from pyspark import SparkContext
import datetime
from csv import reader
import pandas            as pd
import numpy             as np
import matplotlib
import matplotlib.pyplot as plt
import sys #only needed to determine Python version number
%matplotlib inline
matplotlib.style.use('ggplot')

### Data Clean

In [2]:
sc = SparkContext()
# use dataclean.py from here to clean data
filepath    = r'./../../../NYPD_Complaint_Data_Historic.csv'
df = sc.textFile(filepath).map(lambda l: reader([l]).__next__())

header = df.first() # csv header

df = df.filter(lambda line: line != header).cache() # filter out header

In [13]:
def transform(array):
    array[1] = datetime.datetime.strptime(array[1], '%m/%d/%Y')
    array[2] = datetime.datetime.strptime(array[2], '%H:%M:%S').time()
    return array

#replace year 1015
def replaceYear2015to2015(array):
    array[1] = array[1].replace('1015', '2015')
    return array

def replaceTime24to00(array):
    array[2] = array[2].replace('24:00:00', '00:00:00')
    return array

t = df \
    .filter(lambda array: array[1] != '' and array[2] != '') \
    .map(replaceYear2015to2015) \
    .map(replaceTime24to00) \
    .map(transform) \
    .filter(lambda array: (array[1].year >= 2006 and array[1].year <= 2015))

In [14]:
def transformYearAndCoordinate(x):
    x[21] = float(x[21])
    x[22] = float(x[22])
    return (x[1].year, [x[21],x[22]])

yearWithCoordinate = t \
    .filter(lambda x: x[21] != '' and x[22] != '') \
    .map(transformYearAndCoordinate) \
    .groupByKey()

yearWithCoordinate.collect().saveAsTextFile('yearCrime')

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39969)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39969)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39969)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:39969)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 827, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception 

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:39969)

In [ ]:
def transformMonthAndCoordinate(x):
    x[21] = float(x[21])
    x[22] = float(x[22])
    return (x[1].month, [x[21],x[22]])

monthWithCoordinate = t \
    .filter(lambda x: x[21] != '' and x[22] != '') \
    .map(transformMonthAndCoordinate) \
    .groupByKey()